In [1]:
import leaderbot
import leaderbot as lb
import numpy as np
import matplotlib.pyplot as plt
import texplot
import time
import scipy

In [2]:
data = lb.data.load()

agents = np.array(data['models'])
x = np.array(data['X'])
y = np.array(data['Y'])
n_agents = len(agents)

# Clean empty data
y_sum = np.sum(y, axis=1)
ind_valid = y_sum != 0
x = x[ind_valid]
y = y[ind_valid]

In [3]:
# wins, losses, ties = y[:, 0], y[:, 1], y[:, 2]

# n_wins = np.zeros((n_agents,), dtype=int)
# n_losses = np.zeros((n_agents,), dtype=int)
# n_ties = np.zeros((n_agents,), dtype=int)
# n_games = np.zeros((n_agents,), dtype=int)

# for i in range(n_agents):
#     ind_0 = (x[:, 0] == i)
#     ind_1 = (x[:, 1] == i)
#     n_wins[i] = np.sum(wins[ind_0]) + np.sum(losses[ind_1])
#     n_losses[i] = np.sum(losses[ind_0]) + np.sum(wins[ind_1])
#     n_ties[i] = np.sum(ties[ind_0]) + np.sum(ties[ind_1])

# n_matches = n_wins + n_losses + n_ties
# p_wins = n_wins / n_matches
# p_losses = n_losses / n_matches
# p_ties = n_ties / n_matches

In [9]:
#algs = [davidson, davidson_scaled, davidson_scaled_r, davidson_scaled_rij]
model = lb.DavidsonScaled(data)
model.train()

In [10]:
p_pred = model.infer()

# Evaluate All

In [1]:
import leaderbot as lb
import numpy as np

In [2]:
data = lb.data.load()
training_data, test_data = lb.data.split(data, test_ratio=0.2)

In [26]:
training_data_no_tie = training_data.copy()
test_data_no_tie = test_data.copy()

training_data_no_tie['Y'][:, -1] = 0
test_data_no_tie['Y'][:, -1] = 0

In [3]:
n_tie_factors = 1

models = [
    lb.models.BradleyTerry(training_data),
    lb.models.RaoKupper(training_data, n_tie_factors=n_tie_factors),
    lb.models.Davidson(training_data, n_tie_factors=n_tie_factors)
]

In [13]:
n_tie = 1
n_cov = 2

models = [
    lb.models.BradleyTerryFactor(training_data, n_cov_factors=n_cov),
    lb.models.RaoKupperFactor(training_data, n_cov_factors=n_cov, n_tie_factors=n_tie),
    lb.models.DavidsonFactor(training_data, n_cov_factors=n_cov, n_tie_factors=n_tie)
]

In [27]:
n_tie = 1
n_cov = 2

models = [
    lb.models.BradleyTerry(training_data_no_tie),
    lb.models.BradleyTerryScaled(training_data_no_tie),
    lb.models.BradleyTerryScaledR(training_data_no_tie),
    # lb.models.BradleyTerryScaledRIJ(training_data, n_tie_factors=n_tie),
    lb.models.BradleyTerryFactor(training_data, n_cov_factors=n_cov),
    
    lb.models.RaoKupper(training_data, n_tie_factors=n_tie),
    lb.models.RaoKupperScaled(training_data, n_tie_factors=n_tie),
    lb.models.RaoKupperScaledR(training_data, n_tie_factors=n_tie),
    # lb.models.RaoKupperScaledRIJ(dtraining_data, n_tie_factors=n_tie),
    lb.models.RaoKupperFactor(training_data, n_cov_factors=n_cov, n_tie_factors=n_tie),
    
    lb.models.Davidson(training_data, n_tie_factors=n_tie),
    lb.models.DavidsonScaled(training_data, n_tie_factors=n_tie),
    lb.models.DavidsonScaledR(training_data, n_tie_factors=n_tie),
    # lb.models.DavidsonScaledRIJ(training_data, n_tie_factors=n_tie),
    lb.models.DavidsonFactor(training_data, n_cov_factors=n_cov, n_tie_factors=n_tie)
]

In [28]:
for model in models:

    name = model.__class__.__name__
    if name.endswith('ScaledRIJ'):
        method = 'L-BFGS-B'
    else:
        method = 'BFGS'

    print(f'training {name:<21} ... ', end='')
    model.train()
    print('done.')

training BradleyTerry          ... done.
training BradleyTerryScaled    ... done.
training BradleyTerryScaledR   ... done.
training BradleyTerryFactor    ... done.
training RaoKupper             ... done.
training RaoKupperScaled       ... done.
training RaoKupperScaledR      ... done.
training RaoKupperFactor       ... done.
training Davidson              ... done.
training DavidsonScaled        ... done.
training DavidsonScaledR       ... done.
training DavidsonFactor        ... done.


In [29]:
metrics = lb.evaluate.model_selection(models, train=False, tie=True, report=True)

+-----------------------+---------+--------+--------+--------+---------+
| model                 | # param | NLL    | CEL    | AIC    | BIC     |
+-----------------------+---------+--------+--------+--------+---------+
| BradleyTerry          |     129 | 0.6336 | 0.6336 | 256.73 | 1020.98 |
| BradleyTerryScaled    |     258 | 0.6331 | 0.6331 | 514.73 | 2043.24 |
| BradleyTerryScaledR   |     259 | 0.6331 | 0.6331 | 516.73 | 2051.16 |
| BradleyTerryFactor    |     516 | 0.6327 | 0.6327 | 1030.73 | 4087.74 |
| RaoKupper             |     258 | 0.6520 | 0.6520 | 514.70 | 2043.20 |
| RaoKupperScaled       |     387 | 0.6377 | 0.6377 | 772.72 | 3065.48 |
| RaoKupperScaledR      |     388 | 0.6377 | 0.6377 | 774.72 | 3073.40 |
| RaoKupperFactor       |     645 | 0.6374 | 0.6374 | 1288.73 | 5109.99 |
| Davidson              |     258 | 0.6336 | 0.6336 | 514.73 | 2043.24 |
| DavidsonScaled        |     387 | 0.6332 | 0.6332 | 772.73 | 3065.49 |
| DavidsonScaledR       |     388 | 0.6332 | 0.63

In [30]:
metrics = lb.evaluate.goodness_of_fit(models, train=False, tie=True, report=True)

+-----------------------+----------------------------+--------+--------+
|                       |    Mean Absolute Error     |        |        |
| model                 |   win   loss    tie    all | KLD    | JSD %  |
+-----------------------+----------------------------+--------+--------+
| BradleyTerry          |  6.91   6.91   0.00   4.61 | 0.0180 | 0.5116 |
| BradleyTerryScaled    |  6.05   6.05   0.00   4.03 | 0.0170 | 0.4832 |
| BradleyTerryScaledR   |  6.03   6.03   0.00   4.02 | 0.0169 | 0.4808 |
| BradleyTerryFactor    |  5.21   5.21   0.00   3.48 | 0.0152 | 0.4341 |
| RaoKupper             | 19.75  19.72   1.49  13.66 | 0.0505 | 1.4226 |
| RaoKupperScaled       |  6.21   6.15   1.42   4.59 | 0.0215 | 0.6371 |
| RaoKupperScaledR      |  6.21   6.16   1.42   4.60 | 0.0215 | 0.6370 |
| RaoKupperFactor       |  5.44   5.36   1.41   4.07 | 0.0207 | 0.6140 |
| Davidson              |  6.91   6.91   0.00   4.61 | 0.0181 | 0.5153 |
| DavidsonScaled        |  6.15   6.14   0.01   4.1

In [31]:
metrics = lb.evaluate.generalization(models, test_data=test_data, train=False, tie=True, report=True)

+-----------------------+----------------------------+--------+--------+
|                       |    Mean Absolute Error     |        |        |
| model                 |   win   loss    tie    all | KLD    | JSD %  |
+-----------------------+----------------------------+--------+--------+
| BradleyTerry          |  7.05   7.05   0.00   4.70 | 0.0189 | 0.5691 |
| BradleyTerryScaled    |  6.54   6.54   0.00   4.36 | 0.0181 | 0.5474 |
| BradleyTerryScaledR   |  6.59   6.59   0.00   4.39 | 0.0182 | 0.5501 |
| BradleyTerryFactor    |  7.12   7.12   0.00   4.75 | 0.0204 | 0.5927 |
| RaoKupper             | 21.02  20.89   1.59  14.50 | 0.0493 | 1.4119 |
| RaoKupperScaled       |  6.95   6.72   1.49   5.05 | 0.0238 | 0.7302 |
| RaoKupperScaledR      |  6.98   6.76   1.49   5.08 | 0.0237 | 0.7283 |
| RaoKupperFactor       |  7.00   6.87   1.48   5.12 | 0.0238 | 0.7312 |
| Davidson              |  7.02   7.02   0.00   4.68 | 0.0186 | 0.5611 |
| DavidsonScaled        |  6.76   6.75   0.01   4.5